# Part Composers

This tutorial introduces <i>part composers</i> -- python [generators](https://www.tutorialsteacher.com/python/python-generator#:~:text=A%20generator%20is%20a%20special,rather%20than%20a%20return%20statement.) that execute independantly in a scheduling queue to add musical data to a score.

Running this notebook requires a jupyter kernel that contains the musx package.  See [INSTALL.md](https://github.com/musx-admin/musx/blob/main/INSTALL.md) for directions on how to install musx in your environment.
<hr style="height:1px;color:gray">

Python imports:

In [2]:
import sys 
sys.path.append('/Users/taube/Software/musx')
from musx import Score, odds, version
print(f"musx.version: {version}")

musx.version: N.N.N


The code for a composer generally follows the format shown in this schematic:

<pre><b>def</b> composer(<i>score</i>, <i>[...]</i>):
    <i>[initializations...]</i>
    <b>for</b> ... :
        <i>[runtime statements...]</i>
        score.add(...)
        <b>yield</b> <i>waittime</i>
    <i>[finalizations...]</i>
</pre>

A composer must provide at least one input parameter (named *score* in this example) to receive a musx Ccore object passed to it by the scheduler. Any number of additional parameters may follow the score parameter according to the needs of the designer. The second line indicates that when the composer is first called it  allocates whatever internal state it needs in order to add content to the score. The main body of the composer is a loop that iterates each time the composer is called by the scheduler. The yield statement is crucial as it literally defines the composer to be a python generator. The value that the generator yields back to the scheduler will be the *time delta* (in seconds) that the scheduler will wait before calling the composer again. If the composer stops yielding (i.e. the loop has ended), or if the yield value is negative, then the loop will terminate.  Once the loop is done, the composer can perform optional finalizations that "clean up" or take actions just before the composer is garbage collected.

Many part composers can run at the same time inside the scheduler, and can *sprout* (add) other composers into the scheduler dynamically, as needed. The best way to think about a composer is that
it represents a unique *time line* that executes to add its unique material to the musical score. As such, it is always evaluated within the context of the score's *scheduler*, a time-based priority queue that represents the flow of musical time in the composition. The scheduler also provides several attributes that composers can access to find out the current runtime state of the score. 

Here is a simple first example of defining a composer and running it in a score:

In [ ]:
score = musx.Score()

def simp(score, ident, length, rhythm):
    # initialization statements
    print(f'simp {ident}: Hiho! scoretime = {score.now}')
    # for loop executes actions and yields (waits) for its next runtime
    for _ in range(length):
        # our for loop just prints a message each time it runs,
        # it doesnt add anything to a score
        print(f'simp {ident}: Running... scoretime = {score.now}')
        # the yield statement is required, it tells the scheduler how long to
        # wait before running the for composer's loop again.
        yield rhythm
    # post processing statements    
    print(f'simp {ident}: Tata! scoretime = {score.now}')

# Run simp in the score's scheduler, passing the composer a string name and values for 
# the number of times is will execure and its rhythm:
score.compose( simp(score, 'a', 3, 1.125) )

print(f"simp: {simp}")

Here is an explanation of the example one line at a time:

------
```score = musx.Score()```

The variable `score` is set to an instance of a musx Score. A score contains a *scheduler*, which acts like a conductor: it manages musical time and ensures that composer functions execute at their correct times in the score.

-----
```def simp(score, ident, length, rhythm):```

A composer function has a name and must define at least one input parameter to receive the active score object. Remaining parameters, if any, provide the initial input for the composer to access before it starts to run. 

-----
```    print(f'simp {ident}: Hiho! scoretime = {score.now}')```

An initialization statement. It will execute one time.

-----
```    for _ in range(length):```

Music composition is a process that starts and ends. This can be reflected in different ways, the most common method is to iterate using a <b>for</b> or <b>while</b> loop. For example, this part composer accepts a length parameter that limits the number of times it executes.

-----
```        print(f'{ident}: scoretime = {score.now}')```

The print statement is the only 'action' that this composer performs. Each time the composer executes the print statement will display the composer's name and the current score time.  The score time is managed by the scheduler, which contains other dynamic state useful to the compositional process, such as the ability to add score data and insert new composers into the composition as part of its workflow.

-----
```        yield rhythm```

The yield statement is crucial: it defines the composer as a Python generator. The value yielded is the time increment, in seconds, until the composer runs again. If the composer stops yielding, or if the yield value is negative, then the composer is deallocated by the scheduler.

-----
```print(f'simp {ident}: Tata! scoretime = {score.now}')```

Finialization statements can execute after the composer's last yield. Even though the composer won't run again these statements can still sprout new composers in the score.

-----
```score.compose(simp(q, 'a', 10, .25))```

The score's compose() method starts the compositional process running. As we will see in the next example, it is possible to simultaneousy add multiple composers to the score at different start times in the composition.


### Multiple composers

In this example three separate instances of the simp() composer generate events at different rates to create three different time lines in the score. When you run this example note that at time 0, 1, 3 and 4 -- where the composers all share a common time point -- they are executing in the same order that they were added to the scheduler (a,b,c).

To run multiple composers in a Scheduler put them in a python list and pass the list to the compose() function: 

In [ ]:
threesimps = [simp(score,'a',5,1), simp(score,'b',10,1/2), simp(score,'c',15,1/3)]
score.compose(threesimps)

### Specifying different start times

To add a composer to a score at a time later than time 0, specify the start time together with the composer as a two element list: [*start*, *composer*]. 

This example uses a python [comprehension](https://www.w3schools.com/python/python_lists_comprehension.asp) to pass four composers and their start times to the score. Each composer starts 2 seconds later than the preceding one, similar to a canon:

In [ ]:
score.compose([ [t*2, simp(score, t, 4, .25)] for t in range(4) ])

### Adding composers dynamically

A composer can create and add new composers dynamically, as part of its workflow while it is running. Run this next example several times -- each time `sprouter()` executes, it has a 50% probability of adding a new composer to the score with a start time 5 seconds into the future from the parent's current run time.

In [ ]:
def sprouter(score, length, rhythm):
    for i in range(length):
        print(f'sprouter: Running... scoretime = {score.now}')
        if musx.odds(.5):
            score.compose([5, simp(score, i, 5, .1)])
        yield rhythm

score.compose([sprouter(score, 5, 1), [10, sprouter(score, 3, 2)]])

## Compositional strategies with composers

Composers are very flexible and can be utilized in a variety of ways:

* A single composer can be called multiple times to create different versions of itself (e.g. same algorithm but with different initial states)
* There is no limit to the number of composers that are simultaneously running.
* A composer can at any point in its lifetime 'sprout' new composers to start at the current time or in the future.

## For more information...

The demos and tutorials directories contains many examples of making music with part composers. See [INSTALL.md](https://github.com/musx-admin/musx/blob/main/INSTALL.md) for more information.